In [ ]:
pip install earthengine-api

In [ ]:
pip install earthengine-api --upgrade

In [ ]:
pip install earthpy

In [ ]:
pip install geehydro

In [7]:
import ee
import datetime

import earthpy as ep

from glob import glob
import geehydro
import pandas as pd

import folium

import earthpy.spatial as es

import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.plot import show
from rasterio.plot import reshape_as_raster, reshape_as_image

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go

from IPython.display import Image

In [8]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=LK9cu9EYc2T1xqTGHuAXfdjnfOpiTinxy8v-lWOCorA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjdBXb4wPMXPQ-ppHnUQ9Qg9G0Ncypp7fPr3gWbTbU1U7BXFQ5bVmQ

Successfully saved authorization token.


In [9]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

## Case Study 1: Ituna/Itata Indigenous Land in Brazil

In [10]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[-4.06738, -52.034], zoom_start= 15)
Ituna_map

In [11]:
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])

In [12]:
# filter area
# Landsat 8 available from 2013-03-18 forwards
landsat_AOI = landsat.filterBounds(Ituna_AOI).filterDate('2013-03-28','2021-12-31')

In [13]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 116


In [14]:
# Names of Landsat 8 Bands
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [15]:
from datetime import datetime as dt

# choose the least cloudy image as example
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0


'2017-07-20 13:36:07'

In [16]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

In [17]:
## If you wanted to export an image
'''
task = ee.batch.Export.image.toDrive(image=imgdown,
                                     description='Brazil Image',
                                     scale=30,
                                     fileNamePrefix='my_export_brazil',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()
'''

"\ntask = ee.batch.Export.image.toDrive(image=imgdown,\n                                     description='Brazil Image',\n                                     scale=30,\n                                     fileNamePrefix='my_export_brazil',\n                                     crs='EPSG:4326',\n                                     fileFormat='GeoTIFF')\ntask.start()\n"

In [18]:
# Check the status of downloading the image
'''
task.status()
'''

'\ntask.status()\n'

### Running NDVI on the least cloudy image

In [19]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])

In [20]:
# Visualize NDVI results
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Ituna_AOI}
Ituna_map.addLayer(ndvi, ndvi_parameters)
Ituna_map

##### Obtaining Yearly Average NDVI Results for this Area

In [21]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2013,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Ituna_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

       NDVI  year
0  0.320958  2013
1  0.518476  2014
2  0.742392  2015
3  0.736439  2016
4  0.713168  2017
5  0.723214  2018
6  0.816954  2019
7  0.650084  2020
8  0.559372  2021


## Case Study 2: Seattle area

In [22]:
# the Seattle area in USA.
Seattle_map = folium.Map(location=[47.6062, -122.3321], zoom_start=12)#47.608013, -122.335167], zoom_start= 12)
Seattle_map

In [23]:
# setting the Area of Interest (AOI)
Seattle_AOI = ee.Geometry.Rectangle([-122.445465, 47.712397, 
                                   -122.253510, 47.542795])

In [24]:
# filter area
landsat_AOI = landsat.filterBounds(Seattle_AOI).filterDate('2013-01-01','2021-12-31')

In [25]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 320


In [26]:
from datetime import datetime as dt

# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0.06


'2016-08-19 19:01:56'

In [27]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Seattle_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

##### NDVI on Seattle

In [28]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])

In [29]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Seattle_AOI}
Seattle_map.addLayer(ndvi, ndvi_parameters)
Seattle_map

In [30]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2013,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Seattle_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

       NDVI  year
0  0.135939  2013
1  0.104162  2014
2  0.157379  2015
3  0.110165  2016
4  0.111646  2017
5  0.112831  2018
6  0.142973  2019
7  0.135991  2020
8  0.127688  2021


# Landsat 7 

Landsat 7 offers data from 1999-2022. We attempt it on the Seattle area here. 

In [51]:
older_landsat = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")

# LANDSAT/LE07/C02/T1_L2 === 1999-2022
# LANDSAT/LT05/C02/T1_L2 === 1984-2012

In [52]:
# the Seattle area in USA.
Seattle_map = folium.Map(location=[47.6062, -122.3321], zoom_start=12)#47.608013, -122.335167], zoom_start= 12)
Seattle_map

In [53]:
# setting the Area of Interest (AOI)
Seattle_AOI = ee.Geometry.Rectangle([-122.445465, 47.712397, 
                                   -122.253510, 47.542795])

In [54]:
# filter area
landsat_AOI = older_landsat.filterBounds(Seattle_AOI).filterDate('1999-01-01','2021-12-31')
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'sr_atmos_opacity',
 'sr_cloud_qa',
 'pixel_qa',
 'radsat_qa']

In [55]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 691


In [56]:
from datetime import datetime as dt

# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0


'2000-09-25 18:46:05'

In [58]:
parameters = {'min': 0,
              'max': 1500,
              'dimensions': 512,
              'bands': ['B3', 'B2', 'B1'],
              'region': Seattle_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

In [59]:
ndvi = least_cloudy.normalizedDifference(['B4', 'B3'])

In [60]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Seattle_AOI}
Seattle_map.addLayer(ndvi, ndvi_parameters)
Seattle_map

In [62]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B4', 'B3']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(1999,2012)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Seattle_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

        NDVI  year
0   0.108557  1999
1   0.041191  2000
2   0.106721  2001
3   0.071706  2002
4   0.065997  2003
5  -0.004023  2004
6   0.096524  2005
7   0.090295  2006
8   0.135222  2007
9   0.056991  2008
10  0.106148  2009
11  0.074374  2010
12  0.079690  2011
13  0.142582  2012


# Landsat 5

Landsat 5 offers data from 1984-2011. We attempt it on the Seattle area here. 

In [63]:
older_landsat = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")

# LANDSAT/LE07/C02/T1_L2 === 1999-2022
# LANDSAT/LT05/C02/T1_L2 === 1984-2012

In [64]:
# the Seattle area in USA.
Seattle_map = folium.Map(location=[47.6062, -122.3321], zoom_start=12)#47.608013, -122.335167], zoom_start= 12)
Seattle_map

In [65]:
# setting the Area of Interest (AOI)
Seattle_AOI = ee.Geometry.Rectangle([-122.445465, 47.712397, 
                                   -122.253510, 47.542795])

In [66]:
# filter area
landsat_AOI = older_landsat.filterBounds(Seattle_AOI).filterDate('1984-03-16','2012-05-05')

In [67]:
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'sr_atmos_opacity',
 'sr_cloud_qa',
 'pixel_qa',
 'radsat_qa']

In [68]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 734


In [69]:
from datetime import datetime as dt

# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0


'1985-08-23 18:25:12'

In [70]:
parameters = {'min': 0,
              'max': 1500,
              'dimensions': 512,
              
              'bands': ['B3', 'B2', 'B1'],
              'region': Seattle_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

In [71]:
ndvi = least_cloudy.normalizedDifference(['B4', 'B3'])

In [72]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Seattle_AOI}
Seattle_map.addLayer(ndvi, ndvi_parameters)
Seattle_map

In [74]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B4', 'B3']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(1984, 2011)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Seattle_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

        NDVI  year
0   0.166556  1984
1   0.152402  1985
2   0.139088  1986
3   0.148032  1987
4   0.168761  1988
5   0.156491  1989
6   0.141143  1990
7   0.165150  1991
8   0.151263  1992
9   0.125600  1993
10  0.129215  1994
11  0.123545  1995
12  0.140742  1996
13  0.153685  1997
14  0.136079  1998
15  0.133730  1999
16  0.145830  2000
17  0.168834  2001
18  0.135019  2002
19  0.098150  2003
20  0.124544  2004
21  0.157577  2005
22  0.143561  2006
23  0.109133  2007
24  0.174670  2008
25  0.148121  2009
26  0.087233  2010
27  0.154014  2011
